# Airline Revenue Management

## Description
One potential application of reinforcement learning involves managing available seats in different classes on different airplanes to maximize revenue.

The airline revenue management problem addresses the problem by modeling an environment where there are multiple locations with flights to and from each. Each flight has the same number of seats and there is a deadline to make all bookings. Customers can arrive and make bookings on the flights, but the agent can choose whether or not to accept the customer. The goal of the agent is to choose when to accept customers to maximize revenue.

At each time step, a customer will arrive or no customer arrives at all. Based on the action, the customer's booking is either accepted or declined. This is then reflected by updating the number of available seats on each flight that is affected by the booking and by calculating the reward, which represents the revenue earned by accepting (or declining) the booking.

## Model Assumptions

## Dynamics

### State Space
The state space is the set of all possible available seats for every flight into and out of each location up to the full capacities. 
$ S = R_{L_1} \times R_{L_2} \times ... \times R_{L_k} $ where $k$ is the total number flights.

### Action Space
The action space is all possible binary vectors of length n which tells you whether a
customer (with a specific fare and itinerary) is accepted or declined by the airline company.

### Reward
The one-step reward is the revenue gained from applying the predetermined action (of this
time-step) to a customer who appears during this time-step.


## Environment

## Heuristic Agents